In [27]:
import pandas as pd
import re

In [28]:
# Creating Participant dataframe
# Removing unecessary rows/cols and labelling group 

r_df = pd.read_csv('data.csv')
keep_cols = ['Code', 'aoa_eng', 'L1', 'par_chi_can_age']
r_df = r_df.loc[:, keep_cols]
r_df = r_df[~r_df['Code'].isin([797, 702])]
r_df['Group'] = ['Monolingual', 'Early', 'Late', 'Early', 'Late', 'Early', 'Late', 'Late']
r_df = r_df.sort_values(by='Group').reset_index(drop=True)
#r_df

In [29]:
# Initialize error table for errant annotations
error_types = {
    # 'code': [],
    # 'group': [],
    'ADJ': [],
    'ADJ:FORM': [],
    'ADV': [],
    'CONJ': [],
    'CONTR': [],
    'DET': [],
    'MORPH': [],
    'NOUN': [],
    'NOUN:INFL': [],
    'NOUN:NUM': [],
    'NOUN:POSS': [],
    'ORTH': [],
    'OTHER': [],
    'PART': [],
    'PREP': [],
    'PRON': [],
    'PUNCT': [],
    'SPELL': [],
    'UNK': [],
    'VERB': [],
    'VERB:FORM': [],
    'VERB:INFL': [],
    'VERB:SVA': [],
    'VERB:TENSE': [],
    'WO': []
}

err_df = pd.DataFrame(error_types)
# integer vals only
err_df = err_df.astype(int)
err_df

,ADJ,ADJ:FORM,ADV,CONJ,CONTR,DET,MORPH,NOUN,NOUN:INFL,NOUN:NUM,NOUN:POSS,ORTH,OTHER,PART,PREP,PRON,PUNCT,SPELL,UNK,VERB,VERB:FORM,VERB:INFL,VERB:SVA,VERB:TENSE,WO


In [30]:
# Merging errant table and participant table

merged_df = pd.concat([r_df, err_df],axis=0, ignore_index=True).fillna(0)
numerical_cols = merged_df.select_dtypes(include=['number'])
merged_df[numerical_cols.columns] = numerical_cols.astype(int)
#merged_df

In [31]:
# # create rows in the error table according to which participants we use
# for code_num in df['code']:
#     if not (err_df['code'] == code_num).any():
#         new_row = {'code': code_num}
#         cor_group = r_df.loc[df['Code'] == code_num, 'group'].values[0]
#         new_row['group'] = cor_group
#         for column in err_df.columns[2:]:
#             new_row[column] = 0
#         err_df = pd.concat([err_df, pd.DataFrame([new_row])], ignore_index=True)
# err_df

In [32]:
# for each participant code, assign error values to that code
for code_num in merged_df['Code']:
    # change input file for model vs manual annotations
    #input_file = f'm2/{code_num}.txt'
    input_file = f'hand-cor/{code_num}.txt'
    row_index = merged_df.index[merged_df['Code'] == code_num]
    with open(input_file, 'r') as i_file:
            for line in i_file:
                if line[0] == 'A':
                    # regex to find error type
                    pattern = r'\|\|\|([A-Z]:)?([^|]+?)\|\|\|'
                    match = re.search(pattern, line)

                    # extract the matched text
                    if match:
                        captured_text = match.group(2)
                        #print(captured_text)
                        if captured_text == 'noop':
                            pass
                        elif captured_text == 'REQUIRED':
                             print(input_file, line)
                        else:
                            merged_df.loc[row_index, captured_text] += 1
                            #print(captured_text)
                    else:
                        print("No match found")
                else:
                    pass
merged_df = merged_df.sort_values(by='Group').reset_index(drop=True)
pd.set_option("display.max_columns", None) 
# change for model vs manual annotations
# merged_df.to_excel('results.excel')
merged_df.to_csv('true_results.csv')
#merged_df